In [1]:
import tensorflow as tf
import numpy as np

IMAGE_SIZE = 28
NUM_CHANNELS = 1
PIXEL_DEPTH = 255
NUM_LABELS = 10
VALIDATION_SIZE = 10000  # Size of the validation set.
SEED = 66478  # Set to None for random seed.
BATCH_SIZE = 64
NUM_EPOCHS = 10
EVAL_BATCH_SIZE = 64
EVAL_FREQUENCY = 100

(train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.mnist.load_data()

validation_data = train_data[:VALIDATION_SIZE, ...]
validation_labels = train_labels[:VALIDATION_SIZE]
train_data = train_data[VALIDATION_SIZE:, ...]
train_labels = train_labels[VALIDATION_SIZE:]

# # Reshaping the array to 4-dims so that it can work with the Keras API
train_data = train_data.reshape(train_data.shape[0], train_data.shape[1], train_data.shape[2], 1)
validation_data = validation_data.reshape(validation_data.shape[0], validation_data.shape[1], validation_data.shape[2], 1)
test_data = test_data.reshape(test_data.shape[0], test_data.shape[1], test_data.shape[2], 1)

# Making sure that the values are float so that we can get decimal points after division
train_data = train_data.astype('float32')
validation_data = validation_data.astype('float32')
test_data = test_data.astype('float32')
        
# Normalizing the RGB codes by dividing it to the max RGB value.
train_data /= 255
validation_data /=255
test_data /= 255


In [2]:
train_data.shape

(50000, 28, 28, 1)

In [3]:
def error_rate(predictions, labels):
    """Return the error rate based on dense predictions and sparse labels."""
    return 100.0 - 
        (100.0 * np.sum(np.argmax(predictions, 1) == labels) / predictions.shape[0])

In [4]:
def model(data, train=False):
    """The Model definition."""
    # 2D convolution, with 'SAME' padding (i.e. the output feature map
    # has the same size as the input). Note that {strides} is a 4D array
    # whose shape matches the data layout: [image index, y, x, depth].
    conv = tf.nn.conv2d(data,
                        conv1_weights,
                        strides=[1, 1, 1, 1],
                        padding='SAME')
    # Bias and rectified linear non-linearity.
    relu = tf.nn.relu(tf.nn.bias_add(conv, conv1_biases))
    # Max pooling. The kernel size spec {ksize} also follows the layout 
    # of the data. Here we have a pooling window of 2, and a stride of
    # 2.
    pool = tf.nn.max_pool(relu,
                          ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1],
                          padding='SAME')
    conv = tf.nn.conv2d(pool,
                        conv2_weights,
                        strides=[1, 1, 1, 1],
                        padding='SAME')
    relu = tf.nn.relu(tf.nn.bias_add(conv, conv2_biases))
    pool = tf.nn.max_pool(relu,
                        ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1],
                        padding='SAME')
    # Reshape the feature map cuboid into a 2D matrix to feed it to the
    # fully connected layers.
    pool_shape = pool.get_shape().as_list()
    reshape = tf.reshape( pool, [pool_shape[0], pool_shape[1] * pool_shape[2] * pool_shape[3]])
    # Fully connected layer. Note that the '+' operation automatically
    # broadcasts the biases.
    hidden = tf.nn.relu(tf.matmul(reshape, fc1_weights) + fc1_biases)
    # Add a 50% dropout during training only. Dropout also scales
    # activations such that no rescaling is needed at evaluation time.
    if train:
        hidden = tf.nn.dropout(hidden, 0.5, seed=SEED)
    return tf.matmul(hidden, fc2_weights) + fc2_biases

In [5]:
num_epochs = NUM_EPOCHS
train_size = train_labels.shape[0]

In [6]:
# This is where training samples and labels are fed to the graph.
# These placeholder nodes will be fed a batch of training data at each
# training step using the {feed_dict} argument to the Run() call below.
train_data_node = tf.placeholder(
    tf.float32,
    shape=(BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))
train_labels_node = tf.placeholder(tf.int64, shape=(BATCH_SIZE,))
eval_data = tf.placeholder(
    tf.float32,
    shape=(EVAL_BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))

In [7]:
# The variables below hold all the trainable weights. They are passed
# an initial value which will be assigned when we call:
# {tf.global_variables_initializer().run()}
conv1_weights = tf.Variable(
    # 5x5 filter, depth 32.
    tf.truncated_normal([5, 5, NUM_CHANNELS, 32],  
                        stddev=0.1,
                        seed=SEED, dtype=tf.float32))
conv1_biases = tf.Variable(tf.zeros([32], dtype=tf.float32))
conv2_weights = tf.Variable(
    tf.truncated_normal([5, 5, 32, 64], 
                        stddev=0.1,
                        seed=SEED, dtype=tf.float32))
conv2_biases = tf.Variable(tf.constant(0.1, shape=[64],
                           dtype=tf.float32))
# fully connected, depth 512.
fc1_weights = tf.Variable(tf.truncated_normal([IMAGE_SIZE // 4 * IMAGE_SIZE // 4 * 64, 
                                               512],
                        stddev=0.1,
                        seed=SEED,
                        dtype=tf.float32))
fc1_biases = tf.Variable(tf.constant(0.1, shape=[512],
                         dtype=tf.float32))
fc2_weights = tf.Variable(tf.truncated_normal([512, NUM_LABELS],
                                              stddev=0.1,
                                              seed=SEED,
                                              dtype=tf.float32))
fc2_biases = tf.Variable(tf.constant(0.1, shape=[NUM_LABELS], 
                                     dtype=tf.float32))


In [8]:
# Training computation: logits + cross-entropy loss.
logits = model(train_data_node, True)
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
    labels=train_labels_node, logits=logits))

# L2 regularization for the fully connected parameters.
regularizers = (tf.nn.l2_loss(fc1_weights)
                + tf.nn.l2_loss(fc1_biases)
                + tf.nn.l2_loss(fc2_weights)
                + tf.nn.l2_loss(fc2_biases))
# Add the regularization term to the loss.
loss += 5e-4 * regularizers

In [9]:
# Optimizer: set up a variable that's incremented once per batch and
# controls the learning rate decay.
batch = tf.Variable(0, dtype=tf.float32)
# Decay once per epoch, using an exponential schedule starting at 0.01.
learning_rate = tf.train.exponential_decay(
    0.01,                # Base learning rate.
    batch * BATCH_SIZE,  # Current index into the dataset.
    train_size,          # Decay step.
    0.95,                # Decay rate.
    staircase=True)
# Use simple momentum for the optimization.
optimizer = tf.train.MomentumOptimizer(learning_rate,
                                       0.9).minimize(loss,
                                                     global_step=batch)


In [10]:
# Predictions for the current training minibatch.
train_prediction = tf.nn.softmax(logits)

# Predictions for the test and validation, which we'll compute less
# often.
eval_prediction = tf.nn.softmax(model(eval_data))


In [11]:
# Small utility function to evaluate a dataset by feeding batches of
# data to {eval_data} and pulling the results from {eval_predictions}.
# Saves memory and enables this to run on smaller GPUs.
def eval_in_batches(data, sess):
    """Get predictions for a dataset by running it in small batches."""
    size = data.shape[0]
    if size < EVAL_BATCH_SIZE:
        raise ValueError("batch size for evals larger than dataset: %d" 
                         % size)
    predictions = np.ndarray(shape=(size, NUM_LABELS),
                                dtype=np.float32)
    for begin in xrange(0, size, EVAL_BATCH_SIZE):
        end = begin + EVAL_BATCH_SIZE
        if end <= size:
            predictions[begin:end, :] = sess.run(
                eval_prediction,
                feed_dict={eval_data: data[begin:end, ...]})
        else:
            batch_predictions = sess.run(
                eval_prediction,
                feed_dict={eval_data: data[-EVAL_BATCH_SIZE:, ...]})
            predictions[begin:, :] = batch_predictions[begin - size:, :]
    return predictions


In [13]:
import time
import sys
from six.moves import xrange

# Create a local session to run the training.
start_time = time.time()
with tf.Session() as sess:
    # Run all the initializers to prepare the trainable parameters.
    tf.global_variables_initializer().run()
    # Loop through training steps.
    for step in xrange(int(num_epochs * train_size) // BATCH_SIZE):
        # Compute the offset of the current minibatch in the data.
        # Note that we could use better randomization across epochs.
        offset = (step * BATCH_SIZE) % (train_size - BATCH_SIZE)
        batch_data = train_data[offset:(offset + BATCH_SIZE), ...]
        batch_labels = train_labels[offset:(offset + BATCH_SIZE)]
        # This dictionary maps the batch data (as a numpy array) to the
        # node in the graph it should be fed to.
        feed_dict = {train_data_node: batch_data,
                     train_labels_node: batch_labels}
        # Run the optimizer to update weights.
        sess.run(optimizer, feed_dict=feed_dict)
        # print some extra information once reach the evaluation frequency
        if step % EVAL_FREQUENCY == 0:
            # fetch some extra nodes' data
            l, lr, predictions = sess.run([loss, learning_rate,
                                           train_prediction],
                                          feed_dict=feed_dict)
            elapsed_time = time.time() - start_time
            start_time = time.time()
            print('Step %d (epoch %.2f), %.1f ms' %
                  (step, float(step) * BATCH_SIZE / train_size,
                   1000 * elapsed_time / EVAL_FREQUENCY))
            print('Minibatch loss: %.3f, learning rate: %.6f' % (l, lr))
            print('Minibatch error: %.1f%%'
                % error_rate(predictions, batch_labels))
            print('Validation error: %.1f%%' % error_rate(
                eval_in_batches(validation_data, sess), validation_labels))
            sys.stdout.flush()
    # Finally print the result!
    test_error = error_rate(eval_in_batches(test_data, sess),
                            test_labels)
    print('Test error: %.1f%%' % test_error)
    

Step 0 (epoch 0.00), 20.3 ms
Minibatch loss: 8.381, learning rate: 0.010000
Minibatch error: 89.1%
Validation error: 77.6%
Step 100 (epoch 0.13), 2163.0 ms
Minibatch loss: 3.309, learning rate: 0.010000
Minibatch error: 4.7%
Validation error: 7.4%
Step 200 (epoch 0.26), 2336.1 ms
Minibatch loss: 3.267, learning rate: 0.010000
Minibatch error: 7.8%
Validation error: 7.4%
Step 300 (epoch 0.38), 2322.9 ms
Minibatch loss: 3.189, learning rate: 0.010000
Minibatch error: 6.2%
Validation error: 4.0%
Step 400 (epoch 0.51), 2254.3 ms
Minibatch loss: 3.243, learning rate: 0.010000
Minibatch error: 4.7%
Validation error: 3.1%
Step 500 (epoch 0.64), 2234.5 ms
Minibatch loss: 3.122, learning rate: 0.010000
Minibatch error: 4.7%
Validation error: 3.0%
Step 600 (epoch 0.77), 2200.1 ms
Minibatch loss: 2.950, learning rate: 0.010000
Minibatch error: 0.0%
Validation error: 2.7%
Step 700 (epoch 0.90), 2216.1 ms
Minibatch loss: 2.985, learning rate: 0.010000
Minibatch error: 4.7%
Validation error: 2.5%
St

Validation error: 1.2%
Step 6600 (epoch 8.45), 2114.6 ms
Minibatch loss: 1.811, learning rate: 0.006634
Minibatch error: 0.0%
Validation error: 1.2%
Step 6700 (epoch 8.58), 2089.6 ms
Minibatch loss: 1.840, learning rate: 0.006634
Minibatch error: 1.6%
Validation error: 1.2%
Step 6800 (epoch 8.70), 2104.4 ms
Minibatch loss: 1.790, learning rate: 0.006634
Minibatch error: 0.0%
Validation error: 1.3%
Step 6900 (epoch 8.83), 2201.0 ms
Minibatch loss: 1.793, learning rate: 0.006634
Minibatch error: 0.0%
Validation error: 1.2%
Step 7000 (epoch 8.96), 2172.8 ms
Minibatch loss: 1.778, learning rate: 0.006634
Minibatch error: 0.0%
Validation error: 1.1%
Step 7100 (epoch 9.09), 2096.9 ms
Minibatch loss: 1.753, learning rate: 0.006302
Minibatch error: 0.0%
Validation error: 1.0%
Step 7200 (epoch 9.22), 2315.8 ms
Minibatch loss: 1.743, learning rate: 0.006302
Minibatch error: 0.0%
Validation error: 1.2%
Step 7300 (epoch 9.34), 2409.9 ms
Minibatch loss: 1.749, learning rate: 0.006302
Minibatch erro